In [1]:
import sys
import numpy as np
from collections import OrderedDict
from paraview2gmsh import process
import pygmsh as pg

layers = OrderedDict()
BCs = dict()

for i in range(8):
    name = 'layer' + str(i)
    layers[name] = './example2/profile'+str(i)+'.csv'
#layers['l0']='./profile0.csv'
#BCs['layer0'] = 'upper'
#BCs['layer7'] = 'lower'
lcar = 10e3  # Characteristic length

geo = 'example.geo'

mesh = process(layers, BCs, lcar, geo, vscale=1)


Processing ./example2/profile0.csv
> adding points
> creating lines
> creating line loops
> creating surface
> assigning physcial surface name layer0
Processing ./example2/profile1.csv
> adding points
> creating lines
> creating line loops
> creating surface
> assigning physcial surface name layer1
Processing ./example2/profile2.csv
> adding points
> creating lines
> creating line loops
> creating surface
> assigning physcial surface name layer2
Processing ./example2/profile3.csv
> adding points
> creating lines
> creating line loops
> creating surface
> assigning physcial surface name layer3
Processing ./example2/profile4.csv
> adding points
> creating lines
> creating line loops
> creating surface
> assigning physcial surface name layer4
Processing ./example2/profile5.csv
> adding points
> creating lines
> creating line loops
> creating surface
> assigning physcial surface name layer5
Processing ./example2/profile6.csv
> adding points
> creating lines
> creating line loops
> creating

In [2]:
def process(layers, BCs, lcar, geo, vscale=1):
    """
    Start the conversion process.

    Parameters
    ----------
    layers : collections.OrderedDict
        Ordered dictionary with {layername:file}. Should be ordered from top to
        bottom
    BCs : dict
        Defines the layers where boundary conditions are {layername:BC} where
        BC either `upper` or `lower`
    lcar : float
        The characteristic length defining the resolution
    geo : str
        Output file name
    vscale : float
        Scale the data z-axis by this value.
    """
    geom = pg.built_in.Geometry()
    geom.add_raw_code('Geometry.AutoCoherence = 2;')

    line_BC_left = []
    line_BC_right = []
    # Obtain the layer polygons
    for layer in layers.keys():
        if layer in BCs.keys():
            BC = BCs[layer]
        else:
            BC = None
        left, right = add_surface(layers[layer], layer, geom, lcar, BC, vscale)
        line_BC_left.append(left)
        line_BC_right.append(right)
    geom.add_physical_line(line_BC_left, label='Left')
    #geom.add_physical_line(line_BC_right, label='Right')

In [7]:
filename='./example2/profile0.csv'
geom = pg.built_in.Geometry()
vscale=1
vertical_offset=0
BC=None
lcar=1
# Important: columns are loaded in order y,z,x!
body = np.genfromtxt(filename, delimiter=',', skip_header=1,
                     usecols=[1,2,0], names=['y','z','x'])
# Sort so that y-coords are increasing
body.sort(order=['y','z'])
# Convert back to a ndarray
oldshape = body.shape
body = body.view(np.float64).reshape(oldshape+(-1,))
#body = np.loadtxt(filename, delimiter=',', skiprows=1, usecols=[1, 2, 0])
body[:, 1] *= vscale
body[:, 2] = 0
coords_top = body[1::2]
coords_bot = body[::2] - vertical_offset
coords = np.concatenate((coords_top, coords_bot[::-1]))
# Add points
points = []
lines = []

print('> adding points')
for row in coords:
    points.append(geom.add_point(row, lcar))

print('> creating lines')
for i in range(len(points)-1):
    lines.append(geom.add_line(points[i], points[i+1]))
lines.append(geom.add_line(points[-1], points[0]))

line_BC_left = lines[-1]
#line_id = int(len(lines)/2 - 1)
#line_BC_right = lines[line_id]
line_BC_right = None

if BC == 'upper':
    print('> defining upper BC `Top`')
    geom.add_physical_line(lines[:coords_top.shape[0]-1], label='Top')
elif BC == 'lower':
    print('> defining lower BC `Bottom`')
    geom.add_physical_line(lines[coords_top.shape[0]:-1], label='Bottom')

print('> creating line loops')
ll = geom.add_line_loop(lines)

print('> creating surface')
ps = geom.add_plane_surface(ll)

print('> assigning physcial surface name', name)
geom.add_physical_surface(ps, label=name)

> adding points
> creating lines
> creating line loops
> creating surface
> assigning physcial surface name layer7


In [13]:
p0=points[0]
p1=points[-1]

In [14]:
p0.x

array([-3.e+05,  1.e+00,  0.e+00])

In [15]:
p1.x

array([-300000.,   -3397.,       0.])

In [16]:
p0.x-p1.x

array([   0., 3398.,    0.])

In [17]:
np.linalg.norm(p0.x-p1.x)

3398.0

In [28]:
p0, p1 = lines[-1].points